In [2]:
import numpy as np 
import pandas as pd 
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as snsa
from sklearn.linear_model import LinearRegression#sns.set()

In [8]:
df = pd.read_csv('201.csv')
df["admitted"] = df["Admitted"].map({'Yes': 1, 'No': 0})

In [9]:
#Variabels defenieren
x1 = df["SAT"]
y = df["admitted"]

Die Annahmen der Logistischen Regression:
1. Lineare Beziehung zwischen den unabhängigen Variablen und der logit Transformation der abhängigen Variablen
2. Keine Endogenität
3. Störgröße sind nomalverteilt und Kein Homoscedasticity
4. Keine Autokorrelation
5. Keine Multikollinearität


In [11]:
#Regression
x = sm.add_constant(x1)
reg = sm.Logit(y,x)
result = reg.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.137766
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               admitted   No. Observations:                  168
Model:                          Logit   Df Residuals:                      166
Method:                           MLE   Df Model:                            1
Date:                Thu, 20 Mar 2025   Pseudo R-squ.:                  0.7992
Time:                        17:22:06   Log-Likelihood:                -23.145
converged:                       True   LL-Null:                       -115.26
Covariance Type:            nonrobust   LLR p-value:                 5.805e-42
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -69.9128     15.737     -4.443      0.000    -100.756     -39.070
SAT            0.0420      0.009      4.454      0.000       0.024       0.060
==============================================================================

Possibly complete quasi-separation: A fraction 0.27 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

- vorgegeben max Iterations ist 35, das model wird versucht jedes mal eine Regression anzupassen
- function value ist der Wert der Funktion die minimiert wird
- LLNull ist der Log-Likelihood Wert des Modells ohne unabhängige Variabeln
- LLR ist der Log-Likelihood Wert des Modells mit unabhängigen Variabeln
- LLR p-value zeigt wie unterschiedlich LLR und LLNull sind 
- Pseudo R-squ soll inzwischen 0,2 und 0,4 sein

Logit Model Funktion:
$$
\text{logit}(P) = \log\left(\frac{P}{1 - P}\right) = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n
$$

wo:
- $ \log $: Logarithmus.
- $ \frac{P}{1 - P} $: Chance (odds), es ist gleich die Möglichkeit eines Ereignis passiert geteilt durch die Wahrscheinlichkeit, die das Ereignis nicht eintritt 

1 Einheit $X_1$ erhöht/verringert die Chance um das $e^{\beta_1}$-fache



In [12]:
np.exp(0.0420)

1.0428944787507632

1 Einheit SAT erhöht die Chance um 4 Prozent

### Berechnung der genauigkeit

In [15]:
cm_df = pd.DataFrame(result.pred_table())
cm_df.columns = ["Predicted 0", "Predicted 1"]
cm_df = cm_df.rename(index={0: "Actual 0", 1: "Actual 1"})
cm_df

,Predicted 0,Predicted 1
Actual 0,67.0,7.0
Actual 1,7.0,87.0


es ist ein Confusion Matrix

67 mal richtig 0 vorhergesagt 

87 mal richtig 1 vorhergesagt

7 mal falsch 0 (wenn 1 richtig ist) vorhergesagt

7 mal falsch 1 (wenn 0 richtig ist) vorhergesagt

Richtigkeit(Accuracy) = (67+87)/(67+87+7+7)
Fehlklassifizierungsrate (Misclassification Rate) = (7+7)/(67+87+7+7)

In [16]:
cm = np.array(cm_df)
accuracy_train = (cm[0,0]+cm[1,1])/cm.sum() 
accuracy_train

0.9166666666666666

In [ ]:
Methode2 für Confusion Matrix
das erlaubt die confusion matrix mit den test daten zu berechnen. Das Resultat ist die echte accuracy
Die train Datei hat öfters eine höhere accuracy als die test Datei wegen overfitting

In [ ]:
def confusion_matrix(data,actual_values,model):
        
        # Confusion matrix 
        
        # Parameters
        # ----------
        # data: data frame or array
            # data is a data frame formatted in the same way as your input data (without the actual values)
            # e.g. const, var1, var2, etc. Order is very important!
        # actual_values: data frame or array
            # These are the actual values from the test_data
            # In the case of a logistic regression, it should be a single column with 0s and 1s
            
        # model: a LogitResults object
            # this is the variable where you have the fitted model 
            # e.g. results_log in this course
        # ----------
        
        #Predict the values using the Logit model
        pred_values = model.predict(data)
        # Specify the bins 
        bins=np.array([0,0.5,1])
        # Create a histogram, where if values are between 0 and 0.5 tell will be considered 0
        # if they are between 0.5 and 1, they will be considered 1
        cm = np.histogram2d(actual_values, pred_values, bins=bins)[0]
        # Calculate the accuracy
        accuracy = (cm[0,0]+cm[1,1])/cm.sum()
        # Return the confusion matrix and 
        return cm, accuracy
confusion_matrix(x,y,result)